In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
LABEL_NAMES = ['EDU', 'GAR']

DATA_PATH = '/content/drive/MyDrive/domain-helper/'
LOG_PATH = '/content/drive/MyDrive/domain-helper/logs/'

%env TOKENIZERS_PARALLELISM=false

!pip install Sentencepiece
!pip install torch
!pip install transformers
!pip install nlp

from transformers import CamembertForSequenceClassification, CamembertTokenizerFast, Trainer, TrainingArguments
import torch
from nlp import load_dataset, DatasetDict

tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
camembert = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=len(LABEL_NAMES))

dataset = load_dataset('csv', data_files=str(DATA_PATH) + '/combined_csv.csv', split='train' )

# 90% train, 10% test + validation
train_test_valid = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

train_set = train_test_valid_dataset['train']
test_set = train_test_valid_dataset['test']
validation_set = train_test_valid_dataset['valid']

def preprocess(data):
    return tokenizer(data['text'], padding=True, truncation=True)

train_set = train_set.map(preprocess, batched=True,
                          batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

train_set.set_format('torch',
                      columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch',
                     columns=['input_ids', 'attention_mask', 'label'])

batch_size = 8
epochs = 5

warmup_steps = 500
weight_decay = 0.01

training_args = TrainingArguments(
    output_dir=f'{DATA_PATH}sort_results',
    label_names=LABEL_NAMES,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    evaluation_strategy='epoch',
    logging_dir='./logs',
)

trainer = Trainer(
    model=camembert,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

trainer.train()
trainer.evaluate()

env: TOKENIZERS_PARALLELISM=false
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


loading file https://huggingface.co/camembert-base/resolve/main/sentencepiece.bpe.model from cache at /root/.cache/huggingface/transformers/dbcb433aefd8b1a136d029fe2205a5c58a6336f8d3ba20e6c010f4d962174f5f.160b145acd37d2b3fd7c3694afcf4c805c2da5fd4ed4c9e4a23985e3c52ee452
loading file https://huggingface.co/camembert-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/84c442cc6020fc04ce266072af54b040f770850f629dd86c5951dbc23ac4c0dd.8fd2f10f70e05e6bf043e8a6947f6cdf9bb5dc937df6f9210a5c0ba8ee48e959
loading file https://huggingface.co/camembert-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/camembert-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/camembert-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/camembert-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f459e43c5ebb87

Epoch,Training Loss,Validation Loss
1,0.087400,No log
2,0.069100,No log


Saving model checkpoint to /content/drive/MyDrive/domain-helper/sort_results/checkpoint-500
Configuration saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1000
Configuration saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1500
Configuration saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/domain-helper/sort_results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/domain-helper/sort_results/checkpoint-2000
Configu

In [3]:
Y_test = validation_set["label"]
X_test_tokenized = tokenizer(validation_set["text"], padding=True, truncation=True)

!pip install sklearn
import numpy as np

from sklearn.metrics import precision_score

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


validation_dataset = Dataset(X_test_tokenized)

# Make prediction
raw_pred, a, b = trainer.predict(validation_dataset)

# Preprocess raw predictions
Y_pred = np.argmax(raw_pred, axis=1)

print(precision_score(Y_test, Y_pred, average='micro'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=54cc5aa5e8f8cb9f03eb2c6a827c6bbdd509383df8e4fa1fab74102855c15b18
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


***** Running Prediction *****
  Num examples = 1135
  Batch size = 8


0.9885462555066079
